In [3]:
import pandas as pd
import numpy as np

In [167]:
pdb_file = './2DX3.pdb'
f = open(pdb_file, "r")
all_model_list = []
for line in f:
    if line.startswith("MODEL"):
        each_model_list = []

    elif line.startswith("ATOM"):
        each_model_list.append(line.strip())
        N = len(each_model_list)
    elif line.startswith("ENDMDL"):
        all_model_list.append(each_model_list)
f.close()

def move_center(all_model_list):
    q_matrixes_from_center = []
    for MODEL in all_model_list:
        q_array = np.array([])
        weight_q_array = np.array([], dtype = 'float64')
        moment_array = np.array([], dtype = 'float64')
        Total_weight = 0
        for line in MODEL:
            x = line.split()[6]
            y = line.split()[7]
            z = line.split()[8]
            element= line.split()[11]
            q = np.array([x, y, z], dtype = 'float64')
            q_array = np.append(q_array, q)

            if element == 'H':
                weight = 1
            elif element == 'C':
                weight = 12
            elif element == 'N':
                weight = 14
            elif element == 'O':
                weight = 16
            elif element == "S":
                weight = 32
            else:
                print("[[[WARNING]]] Please write the weight of Element:" + element)
            moment = q * weight
            moment_array = np.append(moment_array, moment)
            Total_weight += weight
            weight_q = q * (weight**(1/2))
            weight_q_array = np.append(weight_q_array, weight_q)
        q_matrix = q_array.reshape(int(q_array.shape[0]/3), 3)
        weight_q_matrix=  weight_q_array.reshape(int(weight_q_array.shape[0]/3), 3)
        moment_matrix = moment_array.reshape(int(moment_array.shape[0]/3), 3)
        center = np.array([sum(moment_matrix[:, 0]), sum(moment_matrix[:, 1]), sum(moment_matrix[:, 2])]) / Total_weight
        q_matrix_from_center = q_matrix - center
        q_matrixes_from_center.append(q_matrix_from_center)
    return(q_matrixes_from_center)

def Rotate(Matrix1, Matrix2):
    D = []
    for i in [0, 1, 2]:
        for j in [0, 1,2 ]:
            D.append(np.sum(Matrix1[:, i] * Matrix2[:, j]))
    D = np.array(D)
    D_matrix = D.reshape(3, 3)
    U, S, V = np.linalg.svd(D_matrix, full_matrices=False)
    R = np.dot(U, V)
    Rotated = np.dot(Matrix1, R)
    return(Rotated)

def find_Referrence_structure(all_model_list, times=5):
    for time in range(times):
        if time == 0:
            count = 0
            Rotated_matrix_list = []
            for MODEL in all_model_list:
                count += 1
                q_array = np.array([],dtype = 'float64')
                weight_q_array = np.array([], dtype = 'float64')
                moment_array = np.array([], dtype = 'float64')
                Total_weight = 0
                for line in MODEL:
                    x = line.split()[6]
                    y = line.split()[7]
                    z = line.split()[8]
                    element= line.split()[11]
                    q = np.array([x, y, z], dtype = 'float64')
                    q_array = np.append(q_array, q)
                    if element == 'H':
                        weight = 1
                    elif element == 'C':
                        weight = 12
                    elif element == 'N':
                        weight = 14
                    elif element == 'O':
                        weight = 16
                    elif element == "S":
                        weight = 32
                    else:
                        print("[[[WARNING]]] Please write the weight of Element:" + element)
                    moment = q * weight
                    moment_array = np.append(moment_array, moment)
                    Total_weight += weight
                    weight_q = q * (weight**(1/2))
                    weight_q_array = np.append(weight_q_array, weight_q)
                q_matrix = q_array.reshape(int(q_array.shape[0]/3), 3)
                weight_q_matrix=  weight_q_array.reshape(int(weight_q_array.shape[0]/3), 3)
                moment_matrix = moment_array.reshape(int(moment_array.shape[0]/3), 3)
                center = np.array([sum(moment_matrix[:, 0]), sum(moment_matrix[:, 1]), sum(moment_matrix[:, 2])]) / Total_weight
                if count == 1:
                    base_q_matrix_from_center = q_matrix - center
                    Rotated_matrix_list.append(base_q_matrix_from_center)
                else:
                    q_matrix_from_center = q_matrix - center
                    Rotated = Rotate(q_matrix_from_center, base_q_matrix_from_center)
                    Rotated_matrix_list.append(Rotated)
            Ref = sum(Rotated_matrix_list) / len(all_model_list)
        else:
            index = -1
            for Rotated_matrix in Rotated_matrix_list:
                index += 1
                Rotated = Rotate(Rotated_matrix, Ref)
                Rotated_matrix_list[index] = Rotated
            Ref = sum(Rotated_matrix_list) / len(all_model_list)
    return(Ref)

In [134]:
Ref = find_Referrence_structure(all_model_list, times= 20)

In [137]:
q_matrixes_from_center = move_center(all_model_list)

In [138]:
Rotated_matrixes = []
for q_matrix_from_center in q_matrixes_from_center:
    Rotated = Rotate(q_matrix_from_center, Ref)
    Rotated_matrixes.append(Rotated)

In [203]:
r = Rotated_matrixes[0]

In [204]:
q_ = np.array([r[:,0].mean(), r[:,1].mean(), r[:,2].mean()])

In [205]:
q_

array([-0.15599042, -0.31754703,  0.12672858])

In [206]:
r

array([[ -6.32473597, -12.39691235,   5.23792797],
       [ -5.57674124, -12.39691533,   3.9919248 ],
       [ -6.06674591, -11.24891827,   3.10792357],
       [ -7.22474587, -10.84391897,   3.1959273 ],
       [ -5.65574296, -13.76891733,   3.31892909],
       [ -4.92973859, -14.82991435,   4.14892907],
       [ -5.13174909, -13.70692104,   1.88292669],
       [ -5.31373936, -16.23891597,   3.69493475],
       [ -5.76573246, -12.39690951,   6.0669256 ],
       [ -4.53074033, -12.21991376,   4.23991987],
       [ -6.70474294, -14.06291834,   3.26693437],
       [ -3.85273909, -14.69391374,   4.05592412],
       [ -5.17673423, -14.70391152,   5.20292975],
       [ -5.04274995, -14.71692211,   1.48492922],
       [ -5.82575216, -13.13292337,   1.26792797],
       [ -4.15474952, -13.22492027,   1.87392117],
       [ -4.42873882, -16.87591523,   3.69993284],
       [ -6.06073615, -16.64891462,   4.37393909],
       [ -5.72374366, -16.19591921,   2.68593636],
       [ -5.16174984, -10.75791

In [207]:
r -q_

array([[-6.16874555e+00, -1.20793653e+01,  5.11119938e+00],
       [-5.42075082e+00, -1.20793683e+01,  3.86519622e+00],
       [-5.91075549e+00, -1.09313712e+01,  2.98119499e+00],
       [-7.06875545e+00, -1.05263719e+01,  3.06919872e+00],
       [-5.49975254e+00, -1.34513703e+01,  3.19220050e+00],
       [-4.77374817e+00, -1.45123673e+01,  4.02220049e+00],
       [-4.97575867e+00, -1.33893740e+01,  1.75619811e+00],
       [-5.15774894e+00, -1.59213689e+01,  3.56820617e+00],
       [-5.60974204e+00, -1.20793625e+01,  5.94019702e+00],
       [-4.37474991e+00, -1.19023667e+01,  4.11319128e+00],
       [-6.54875252e+00, -1.37453713e+01,  3.14020579e+00],
       [-3.69674867e+00, -1.43763667e+01,  3.92919554e+00],
       [-5.02074381e+00, -1.43863645e+01,  5.07620117e+00],
       [-4.88675953e+00, -1.43993751e+01,  1.35820064e+00],
       [-5.66976174e+00, -1.28153763e+01,  1.14119939e+00],
       [-3.99875910e+00, -1.29073732e+01,  1.74719259e+00],
       [-4.27274840e+00, -1.65583682e+01

In [208]:
dq = (r - q_)

In [209]:
dq.flatten() / (len(Rotated_matrixes) - 1)** (1/2)

array([-2.05624852e+00, -4.02645511e+00,  1.70373313e+00, -1.80691694e+00,
       -4.02645610e+00,  1.28839874e+00, -1.97025183e+00, -3.64379041e+00,
        9.93731662e-01, -2.35625182e+00, -3.50879064e+00,  1.02306624e+00,
       -1.83325085e+00, -4.48379010e+00,  1.06406683e+00, -1.59124939e+00,
       -4.83745577e+00,  1.34073350e+00, -1.65858622e+00, -4.46312467e+00,
        5.85399369e-01, -1.71924965e+00, -5.30712298e+00,  1.18940206e+00,
       -1.86991401e+00, -4.02645416e+00,  1.98006567e+00, -1.45824997e+00,
       -3.96745558e+00,  1.37106376e+00, -2.18291751e+00, -4.58179043e+00,
        1.04673526e+00, -1.23224956e+00, -4.79212224e+00,  1.30973185e+00,
       -1.67358127e+00, -4.79545483e+00,  1.69206706e+00, -1.62891984e+00,
       -4.79979169e+00,  4.52733546e-01, -1.88992058e+00, -4.27179211e+00,
        3.80399797e-01, -1.33291970e+00, -4.30245774e+00,  5.82397529e-01,
       -1.42424947e+00, -5.51945607e+00,  1.19106809e+00, -1.96824858e+00,
       -5.44378920e+00,  

3.0

In [221]:
Q_matrix = []
for r in Rotated_matrixes:
    q_ = np.array([r[:,0].mean(), r[:,1].mean(), r[:,2].mean()])
    dq = (r - q_)
    dq = dq.flatten() / (len(Rotated_matrixes) - 1)** (1/2)
    Q_matrix.append(dq)
Q_matrix = np.array(Q_matrix)
Q_matrix = Q_matrix.T.reshape(3*N, len(Rotated_matrixes))

In [223]:
Q_matrix[:,0]

array([-2.05624852e+00, -4.02645511e+00,  1.70373313e+00, -1.80691694e+00,
       -4.02645610e+00,  1.28839874e+00, -1.97025183e+00, -3.64379041e+00,
        9.93731662e-01, -2.35625182e+00, -3.50879064e+00,  1.02306624e+00,
       -1.83325085e+00, -4.48379010e+00,  1.06406683e+00, -1.59124939e+00,
       -4.83745577e+00,  1.34073350e+00, -1.65858622e+00, -4.46312467e+00,
        5.85399369e-01, -1.71924965e+00, -5.30712298e+00,  1.18940206e+00,
       -1.86991401e+00, -4.02645416e+00,  1.98006567e+00, -1.45824997e+00,
       -3.96745558e+00,  1.37106376e+00, -2.18291751e+00, -4.58179043e+00,
        1.04673526e+00, -1.23224956e+00, -4.79212224e+00,  1.30973185e+00,
       -1.67358127e+00, -4.79545483e+00,  1.69206706e+00, -1.62891984e+00,
       -4.79979169e+00,  4.52733546e-01, -1.88992058e+00, -4.27179211e+00,
        3.80399797e-01, -1.33291970e+00, -4.30245774e+00,  5.82397529e-01,
       -1.42424947e+00, -5.51945607e+00,  1.19106809e+00, -1.96824858e+00,
       -5.44378920e+00,  

In [226]:
U, S, Vt = np.linalg.svd(Q_matrix.T, full_matrices=False)

In [246]:
np.dot(np.dot(Q_matrix, U), S*(-1/2))

array([ -401.02483138,  -983.24441551,   373.71977082,  -402.05506199,
        -947.92282911,   327.73246211,  -446.37712311,  -874.4138597 ,
         240.73849364,  -517.95987709,  -855.62289219,   237.27280421,
        -378.31643899,  -981.02113707,   338.95051383,  -350.61658834,
       -1011.01067159,   381.32693249,  -378.01295712,  -938.15657504,
         304.36684518,  -350.34510368, -1080.88796063,   372.52191245,
        -393.82379562,  -963.23175928,   396.5111586 ,  -386.50606388,
        -952.03671673,   351.39069376,  -381.66071258, -1017.28472014,
         341.59634042,  -330.3291575 , -1006.19989273,   390.07448691,
        -350.39607975,  -985.95285265,   410.88335364,  -394.84658747,
        -929.18815086,   295.16921984,  -366.28778087,  -941.97977416,
         311.24419221,  -372.69261562,  -912.70251606,   282.11811437,
        -328.44602093, -1094.9321153 ,   381.88793405,  -364.20019717,
       -1104.4839028 ,   372.29649287,  -358.14735348, -1093.03011355,
      

In [240]:
Q_matrix.shape

(888, 10)

In [241]:
U.shape

(10, 10)

In [242]:
S.shape

(10,)

In [250]:
Vt.shape

(10, 888)

In [252]:
S

array([158.77215975,  11.5118584 ,   7.50246436,   6.70177502,
         6.43036142,   4.84461493,   4.12336256,   3.51053074,
         3.27486847,   2.80390331])

In [275]:
U, S, Vt = np.linalg.svd(Q_matrix.T, full_matrices=False)

In [269]:
Q_matrix.T

array([[-2.05624852, -4.02645511,  1.70373313, ...,  3.29273587,
         2.19420876, -0.78697407],
       [-1.81469407, -4.08127909,  1.72510047, ...,  2.70013952,
         2.5066088 , -0.74776663],
       [-1.8848902 , -3.79566081,  1.61399217, ...,  2.71813577,
         1.87634038, -0.09300828],
       ...,
       [-2.23901728, -3.99704374,  1.65647511, ...,  3.20494829,
         2.52872461, -0.87220076],
       [-1.49677161, -3.77703692,  1.5728951 , ...,  3.13639813,
         2.55423455, -0.93436474],
       [-1.25667464, -4.15850934,  0.97921198, ...,  2.96897849,
         2.58850681, -0.62996293]])

In [276]:
np.dot(np.dot(U,np.diag(S)), Vt)

array([[-2.05624852, -4.02645511,  1.70373313, ...,  3.29273587,
         2.19420876, -0.78697407],
       [-1.81469407, -4.08127909,  1.72510047, ...,  2.70013952,
         2.5066088 , -0.74776663],
       [-1.8848902 , -3.79566081,  1.61399217, ...,  2.71813577,
         1.87634038, -0.09300828],
       ...,
       [-2.23901728, -3.99704374,  1.65647511, ...,  3.20494829,
         2.52872461, -0.87220076],
       [-1.49677161, -3.77703692,  1.5728951 , ...,  3.13639813,
         2.55423455, -0.93436474],
       [-1.25667464, -4.15850934,  0.97921198, ...,  2.96897849,
         2.58850681, -0.62996293]])

In [281]:
AV = np.dot(np.dot(Q_matrix, U), np.diag(S))

In [283]:
V = np.dot(np.dot(Q_matrix, U), np.diag(S*(-1/2)))

In [285]:
V.shape

(888, 10)